# 2.3 Curry-Howard correspondence

In this notebook we deal with another striking connection between functional programming and, in principle, a completely unrelated field of knowledge. When algebraic data types were introduced, we established a strong connection with _arithmetics_. Now, it's the turn of _logic_. We will see that programming functionally is akin, in a very close sense, to doing logic. That, when we write a type signature we are writing propositions that have to be proved; and that when we implement that signature with some term expression, we are actually providing a proof of that proposition. Thus, this notebook resolves to show the following correspondences:

$$
\begin{array}{cc}
\mathrm{\bf Computation} & \mathrm{\bf Logic} \\
\hline 
\mathrm{Terms} & \mathrm{Proofs} \\
\mathrm{Types} & \mathrm{Propositions} 
\end{array}
$$

This correspondence between logic and computation is known as the Curry-Howard correspondence. It shows that programming functionally and following the laws of logic is the very same thing, ... and who on earth would want to program in a different way!

### References

[__The Curry-Howard correspondence__](https://en.wikipedia.org/wiki/Curry%E2%80%93Howard_correspondence) Wikpedia. 



# A refresher: arithmetic and types

Recall the correspondence between algebraic data types and arithmetic operators: 

$$ 
\begin{array}{cc}
\mathrm{\bf Scala\ ADTs} & \mathrm{\bf Arithmetic} \\
\hline
\mathtt{Unit} & 1 \\
\mathtt{Nothing} & 0 \\
\mathtt{Either[X, Y]} & x + y \\
\mathtt{Tuple2[X, Y]} & x * y \\
\mathtt{X => Y} & {y^x}  
\end{array}
$$

According to this algebraic interpretation of types, the type 

In [123]:
type A = (Unit, Either[Unit, (Boolean => Either[Unit, Unit])])

defined type A

has exactly `1*(1+(1+1)^2)` values. But not only that: corresponding to the equality `1*(1+(1+1)^2)=5`, the following isomorphism holds: 

In [118]:
def from(a: A): Either[Unit, 
    Either[Unit, Either[Unit, Either[Unit, Unit]]]] = ???
def to(a: Either[Unit, 
    Either[Unit, Either[Unit, Either[Unit, Unit]]]]): A = ???

defined function from
defined function to

# Propositions as types

We put forward now the following correspondence: 

$$ 
\begin{array}{cc}
\mathrm{\bf Scala\ ADTs} & \mathrm{\bf Logic} \\
\hline
\mathtt{Nothing} & \bot \\
\mathtt{Either[P, Q]} & p \vee q \\
\mathtt{Tuple2[P, Q]} & p \wedge q \\
\mathtt{P => Q} & p \rightarrow q 
\end{array}
$$


so that the following type:

```scala
(P => Q, P) => Q
```

corresponds to the following formula of propositional logic: 

$\hspace{0.8cm}(p \rightarrow q) \wedge p \rightarrow q$

Let's get used to read Scala types as propositional formulas with these examples: 

$$
\begin{array}{cc}
\mathrm{\bf Scala\ types} & \mathrm{\bf Propositions} \\
\hline
\mathtt{(P, Either[Q, R])} & p \wedge (q\vee r) \\ 
\mathtt{P => Q => (Q, R)} & p \rightarrow q \rightarrow q \wedge r \\
\mathtt{(P => Nothing) => Nothing} & (p \rightarrow \bot) \rightarrow \bot \\ 
\end{array}
$$

What about logical negation and equivalence? They are commonly regarded as abbreviations: 

* $\neg p$ abbreviates $p \rightarrow \bot$
* $p \leftrightarrow q$ abbreviates $p \rightarrow q \wedge q \rightarrow p$

so, they will be represented in Scala through the following type aliases: 

In [127]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)

defined type Not
defined type <=>

Let's translate the following propositional formulas to Scala types: 


$$ 
\begin{array}{cc}
\mathrm{\bf Logic} & \mathrm{\bf Scala\ ADTs} \\
\hline
p \wedge q \rightarrow r \leftrightarrow p \rightarrow q \rightarrow r & \mathtt{(((P,Q)) => R) <=> (P => Q => R)} \\
\neg (p \wedge q) \rightarrow \neg p \vee \neg q & \mathtt{Not[(P, Q)] => Either[Not[P], Not[Q]]}\\
\neg\neg(p \vee \neg p) & \mathtt{Not[Not[Either[P, Not[P]]]]}\\
p \vee (q \wedge r) \leftrightarrow (p \vee q) \wedge (p \vee r) & 
\mathtt{Either[P, (Q, R)] <=> (Either[P, Q], Either[P, R])}
\end{array}
$$


Note that $p \wedge q \rightarrow r$ translates to `((P,Q)) => R`, and not to `(P, Q) => R`. The former is a `Function1` which receives a single argument of tuple type `(P, Q)`; the latter is a `Function2` which receives two arguments of types `P` and `Q`. 

# Proofs as terms

Right, but this is just a syntactic rewriting. In which sense, for instance, the Scala type

`(P, Either[Q, R]) => Either[(P, Q), (P, R)]`

justifies itself as a proper counterpart of the proposition
$p \wedge (q \vee r) \rightarrow p \wedge q \vee p \wedge r$? The answer is that they behave exactly in the same way: we can implement a program (expression or term) of that type if, and only if, the corresponding proposition is a theorem. Thus, the program is a witness that the proposition can be proved; in fact, we can automatically obtained a proof from that program, and viceversa. Let's start from a possible program:



In [140]:
def program[P, Q, R]: ((P, Either[Q, R])) => Either[(P, Q), (P, R)] = 
    t => t._2 match {
        case Left(q) => Left(t._1, q)
        case Right(r) => Right(t._1, r)
    }

defined function program

How is this program related to a proof of $p \wedge (q \vee r) \rightarrow p \wedge q \vee p \wedge r$? In order to talk about proofs we need a formal system, and our proof system of choice is Gentzen's natural deduction system: 

![natural deduction rules](../images/natdedrules.png)

Now, we can give a proof as follows:

$$
\begin{array}{rlr}
1. & p \wedge (q \vee r) & hypothesis \\
2. &p  & \wedge E \\
3. & q \vee r & \wedge E \\
4a. &q & hypothesis \\
5a. &p \wedge q & \wedge I \\
6a. &(p \wedge q) \vee (p \wedge r) & \vee I \\
4b. &r & hypothesis \\
5b. &p \wedge r & \wedge I \\
6b. &(p \wedge q) \vee (p \wedge r) & \vee I \\
7. &(p \wedge q) \vee (p \wedge r) & \vee E \\
8. &p \wedge (q \vee r) \rightarrow (p \wedge q) \vee (p \wedge r) & \rightarrow I
\end{array}
$$

![proof1](../images/proof1.png)

Apparently, this is 

In [138]:
def proof[P, Q]: P And Q => Q And P = 
    Implies.I{ p1: P And Q => 
        And.I(
            And.ER(p1: P And Q): Q, 
            And.EL(p1: P And Q): P
        ): Q And P 
    }: (P And Q => Q And P)

defined function proof

In [128]:
def proof[P, Q]: P => Q => P = 
    (p: P) => (q: Q) => p : P 

defined function proof

Right, but this is just a syntactic rewriting. In which sense, for instance, the Scala type

`Not[(P, Q)] => Either[Not[P], Not[Q]]`

behaves _exactly_ as the proposition
$\neg (p \wedge q) \rightarrow \neg p \vee \neg q$?



In [ ]:
def proof[P, Q]: Not[(P, Q)] => Either[Not[P], Not[Q]] = 
    n => Left(p => )

In [130]:
object Ax{
    def apply[P](a: P): P = a
}

type ⟶[A, B] = A => B

object ⟶{
    def I[A, B](f: A => B): A => B =
        f
    
    def E[A, B](f: A => B)(a: A): B = 
        f(a)
}

type ⊥ = Nothing 

object ⊥{
    def E[A](a: ⊥): A = a
} 

type ∨[A, B] = Either[A, B]

object ∨{
    def IL[A, B](a: A): A ∨ B = Left(a)

    def IR[A, B](b: B): A ∨ B = Right(b)
    
    def E[A, B, C](f: A => C, g: B => C)(or: A ∨ B): C = 
        or.fold(f, g)
}

type ∧[A, B] = (A, B)

object ∧{
    def I[A, B](a: A, b: B): A ∧ B = (a, b)
    
    def EL[A, B](and: A ∧ B): A = and._1
    
    def ER[A, B](and: A ∧ B): B = and._2
}

defined object Ax
defined type ⟶
defined object ⟶
defined type ⊥
defined object ⊥
defined type ∨
defined object ∨
defined type ∧
defined object ∧

In [131]:
type ¬[A] = A ⟶ ⊥
type ⟷[A, B] = (A ⟶ B) ∧ (B ⟶ A)
type ⟵[A, B] = B ⟶ A

defined type ¬
defined type ⟷
defined type ⟵

In [132]:
type Absurd = ⊥
type Or[A, B] = ∨[A, B] 
type And[A, B] = ∧[A, B]
type Not[A] = ¬[A]
type <=>[A, B] = (A => B, B => A)
val Implies = ⟶
val Or = ∨ 
val And = ∧

defined type Absurd
defined type Or
defined type And
defined type Not
defined type <=>
Implies: ⟶.type = ammonite.$sess.cmd129$Helper$$u27F6$@20f28099
Or: ∨.type = ammonite.$sess.cmd129$Helper$$u2228$@5328c170
And: ∧.type = ammonite.$sess.cmd129$Helper$$u2227$@5f4c86c7

$\vdash X \wedge (X \rightarrow Y) \rightarrow Y$

In [82]:
def proof[X, Y]: (X => Y, X) => Y = 
    //(x, f) => f(x)
    _(_)

defined function proof

In [83]:
def proof[X, Y]: ((X, X => Y)) => Y = 
    (a: (X, X => Y)) => 
        (a._2)(a._1)

defined function proof

In [84]:
def proof[X, Y]: X ∧ (X ⟶ Y) ⟶ Y = 
    ⟶.I{ a: X ∧ (X ⟶ Y) => 
        ⟶.E(∧.ER(a))(∧.EL(a))
    }

defined function proof

$X*(Y+Z) \cong X*Y + X*Z$

$\vdash X \wedge (Y \vee Z) \leftrightarrow X \wedge Y \vee X \wedge Z$

In [85]:
def proof[P, Q, R]: (P ∧ (Q ∨ R)) ⟷ ((P ∧ Q) ∨ (P ∧ R)) = 
    ∧.I(
        ⟶.I{ a: P ∧ (Q ∨ R) => 
            ∨.E((q: Q) => ∨.IL(∧.I(∧.EL(a), q)),
                (r: R) => ∨.IR(∧.I(∧.EL(a), r)))(∧.ER(a))
        },
        ⟶.I{ 
            ∨.E((a: P ∧ Q) => ∧.I(∧.EL(a), ∨.IL(∧.ER(a))),
                (a: P ∧ R) => ∧.I(∧.EL(a), ∨.IR(∧.ER(a))))
        }
    )

defined function proof

In [86]:
def proof[P, Q, R]: (P, Either[Q, R]) <=> Either[(P, Q), (P, R)] = 
    ({ case (p, Left(q)) => Left((p,q))
        case (p, Right(r)) => Right((p,r)) },
    { case Left((p,q)) => (p, Left(q))
        case Right((p,r)) => (p, Right(r)) })

defined function proof

$X^{Y+Z} \cong X^Y * X^Z$

$\vdash (Y \vee Z \rightarrow X) \leftrightarrow (Y \rightarrow X) \wedge (Z \rightarrow X)$

In [87]:
def proof[X, Y, Z]: (Y ∨ Z => X) <=> (Y => X ∧ Z => X) = 
    (??? : (Y ∨ Z => X) => (Y => X ∧ Z => X), 
     ??? : (Y ∨ Z => X) <= (Y => X ∧ Z => X))

defined function proof

In [88]:
def proof[X, Y, Z]: (Either[Y, Z] => X) <=> (Y => X, Z => X) = 
    (f => (y => f(Left(y)), z => f(Right(z))) , 
     { case (f, g) => 
         { case Left(y) => f(y)
           case Right(z) => g(z)
         }
     })

defined function proof

$\vdash P \vee Q \rightarrow \neg P \rightarrow Q$ 

In [89]:
def proof[P, Q]: Either[P, Q] => (P => Nothing) => Q = 
    (e: Either[P, Q]) => (f: P => Nothing) => 
        //e.fold[B](f, identity)
        e match {
            case Left(a: P) => f(a): Q
            case Right(b: Q) => b : Q
        }

defined function proof

In [93]:
def proof[P, Q]: P ∨ Q ⟶ (¬[P] ⟶ Q) = 
    ⟶.I{ or: P ∨ Q => 
        ⟶.I{ n: ¬[P] => 
            ∨.E((p: P) => ⊥.E[Q](⟶.E(n)(p)), 
                Ax[Q])(or)
        }
    }

defined function proof

In [29]:

def proof8[A, B]: Or[A, B] => Or[B, A] = 
    Or.elim[A, B, Or[B, A]](Or.applyR[B, A], Or.applyL[B, A])

// Not and implies

def proof3[A, B]: (A => Not[B]) => (A => B) => Not[A] = 
    i => (i2: A => B) => (a: A) => i(a)(i2(a))

def proof2[A, B]: Not[A] => A => B = 
    n => Not.elim[A, B](n)

def proof4[A, B]: Not[A => B] => Not[B] = 
    n => (b: B) => n((_: A) => b)

// Not and and

def proof5[A, B]: Not[A] => Not[And[A, B]] = 
//    (and: And[A, B]) => n(And.elimL(and))
    n => n compose And.elimL

def proof7[A, B]: Not[And[A, B]] => A => Not[B] = 
    n => (a: A) => (b: B) => n(And(a, b))

def proof11[A]: Not[A And Not[A]] = 
    a => And.elimR(a)(And.elimL(a))

// Not and or

def proof9[A, B]: Not[A Or B] => Not[A] And Not[B] = 
    n => And((a: A) => n(Or.applyL(a)), (b: B) => n(Or.applyR(b)))

def proof10[A, B]: (Not[A] And Not[B]) => Not[A Or B] = 
    and => Or.elim[A, B, Absurd](And.elimL(and), And.elimR(and))



defined function proof8
defined function proof3
defined function proof2
defined function proof4
defined function proof41
defined function proof5
defined function proof7
defined function proof11
defined function proof9
defined function proof10

In [13]:
trait ClassicalLogic{
    trait DN{
        def apply[A](nn: Not[Not[A]]): A
    }
    
    val dn: DN
    
    def proof1[A]: (Not[A] => A) => A =
        f => dn((n: Not[A]) => proof11(And(f(n),n)))
}


defined trait ClassicalLogic

All P are S
All S are R
All P are R

In [15]:
def proof[P, S, R]: ((P => S, S => R)) => P => R = 
    t => (p: P) => t._2(t._1(p))

defined function proof

AG. $\{ r \vee p \rightarrow \neg p, p \rightarrow q, \neg q \vee p\} \vdash \neg p$

In [113]:
def proof[P, Q, R](
    p1: R Or P => Not[P], 
    p2: P => Q, 
    p3: Not[Q] Or P): Not[P] = 
    (p: P) => p1(Right(p))(p) : Absurd

defined function proof

$ \vdash (p \rightarrow (q \rightarrow r)) \rightarrow (p \rightarrow q) \rightarrow p \rightarrow r$

In [38]:
def term[P, Q, R]: (P => (Q => R)) => (P => Q) => (P => R) = 
    (p1: P => Q => R) => 
        (p2: P => Q) => 
            (p: P) => p1(p : P)(p2(p) : Q) : R

defined function term

In [94]:
def proof[P, Q, R]: (P ⟶ (Q ⟶ R)) ⟶ ((P ⟶ Q) ⟶ (P ⟶ R)) = 
    ⟶.I{ p1: P ⟶ (Q ⟶ R) => 
        ⟶.I{ p2: P ⟶ Q => 
            ⟶.I{ p: P =>
                ⟶.E(⟶.E(p1)(p))(⟶.E(p2)(p))
            }
        }
    }

defined function proof

$ \vdash p \rightarrow q \rightarrow p$

In [39]:
def proof[P, Q, R]: P => Q => P = 
    (p: P) => 
        (q: Q) => p : P

defined function proof

In [96]:
def proof[P, Q, R]: P ⟶ (Q ⟶ P) = 
    ⟶.I{ p: P => 
        ⟶.I{ q: Q => 
            p : P
        }
    }

defined function proof

In [98]:
def proof[A, B, C]: (A => (B => C)) => (A Either B => C) = 
    (p: A => B => C) => 
        (e: Either[A, B]) => 
            e.fold(
                (a: A) => ??? : C, 
                (b: B) => ??? : C)

defined function proof

$\vdash \neg \neg \neg p \leftrightarrow \neg p$

In [103]:
def proof[P]: Not[Not[Not[P]]] => Not[P] = 
    (nnnp: ((P => Absurd) => Absurd) => Absurd) => 
        (p: P) => 
            nnnp( (np : P => Absurd) => np(p) : Absurd): Absurd

defined function proof

In [104]:
def proof[P]: Not[Not[Not[P]]] => Not[P] = 
    nnnp => p => nnnp(_(p))

defined function proof

Ejercicio AG.

In [107]:
def proof[P, Q, R, S, T](
    p1: P => Not[R] Or S,
    p2: Not[R] => S And Not[T],
    p3: P => T,
    p4: P => Not[S]): Not[P] = 
    (p: P) => 
        p1(p) match {
            case nr: Not[R] => p4(p)(p2(nr)._1) : Absurd
            case s: S => p4(p)(s) : Absurd
        }

defined function proof

In [112]:
def proof[P, Q, R, S, T](
    p1: P ⟶ (¬[R] ∨ S),
    p2: ¬[R] ⟶ (S ∧ ¬[T]),
    p3: P ⟶ T,
    p4: P ⟶ ¬[S]): ¬[P] = 
    ⟶.I((p: P) => 
        ∨.E(
            (nr: Not[R]) => 
                ⟶.E(⟶.E(p4)(p))(∧.EL(⟶.E(p2)(nr))) : Absurd,
            (s: S) => 
                ⟶.E(⟶.E(p4)(p))(s) : Absurd
        )(⟶.E(p1)(p)))

defined function proof